# Smoke Alarm install model
### this notebook represents the current smoke alarm install model

In [1]:
import pandas as pd
import os
import sys
import missingno as msno
from pathlib import Path
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
 path = Path.cwd().parent.parent


In [3]:
# to allow for all variables to be displayed in jupyter
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
def StandardizeColumnNames(df):
    """
    Standardizes column names
    """
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace(', ', '_')
    df.columns = df.columns.str.replace('-', '_')
    df.columns = df.columns.str.replace('/', '_')
    df.columns = df.columns.str.replace('(', '_')
    df.columns = df.columns.str.replace(')', '_')
    df.columns = df.columns.str.replace(' ', '_')
    print(df.columns)
    return df

### Data

In [5]:
input_loc =  path /'Data'/ 'Master Project Data'
output_loc = path /'Data'/ 'processed'

In [6]:
arc_path = input_loc / 'ARC Preparedness Data.csv'
arc = pd.read_csv(arc_path, 
                  dtype = {'GEOID': str, 'Zip': str})
arc = StandardizeColumnNames(arc)
arc.dropna(inplace = True)
arc.head()

Index(['geoid', 'census_block_group_y', 'census_block_group_x', 'city',
       'state', 'zip', 'county', 'in_home_visit_date',
       'smoke_alarms_installed__9_volt_10_year_dhh_',
       '10_year_and_9_volt_alarms_installed', 'dhh_alarms_installed',
       'pre_existing_alarms', 'pre_existing_alarms_tested_and_working',
       'batteries_replaced', 'fire_escape_plans_made',
       'fire_safety_checklists_completed',
       'additional_hazard_education_conducted', 'additional_hazard_type',
       'people_served', 'youth_served', 'seniors_served',
       'veterans_military_members_and_military_family_members_served',
       'individuals_with_disabilities_access_or_functional_needs_served'],
      dtype='object')


,geoid,census_block_group_y,census_block_group_x,city,state,zip,county,in_home_visit_date,smoke_alarms_installed__9_volt_10_year_dhh_,10_year_and_9_volt_alarms_installed,dhh_alarms_installed,pre_existing_alarms,pre_existing_alarms_tested_and_working,batteries_replaced,fire_escape_plans_made,fire_safety_checklists_completed,additional_hazard_education_conducted,additional_hazard_type,people_served,youth_served,seniors_served,veterans_military_members_and_military_family_members_served,individuals_with_disabilities_access_or_functional_needs_served
12,#_010010205002,32.470418,-86.424166,PRATTVILLE,AL,36066,Autauga,9/9/2016,1,1,0,0.0,0.0,0,1,1,0,None,2,0,0,0,0
18,#_010010208012,32.455173,-86.534591,PRATTVILLE,AL,36067,Autauga,9/30/2019,2,2,0,0.0,0.0,0,1,1,1,Tornadoes,5,0,0,1,0
23,#_010010208021,32.524822,-86.573009,PRATTVILLE,AL,36067,Autauga,5/4/2019,3,2,1,0.0,0.0,0,1,1,1,Other,1,0,0,0,0
24,#_010010208021,32.524822,-86.573009,PRATTVILLE,AL,36067,Autauga,9/27/2019,1,1,0,0.0,0.0,0,1,1,1,Tornadoes,1,0,0,0,0
27,#_010010208023,32.543983,-86.492100,PRATTVILLE,AL,36067,Autauga,12/2/2017,3,3,0,0.0,0.0,0,1,1,0,None,1,0,0,0,0


## EDA  

- remove all houses that don't have a previous smoke detector record 
- Determine the median number of house visist
- Visualize visit distribution 
- use ACS data to determine % of blocks visited 
- determine % blocks visited with >15 visits 

In [7]:
# trim geoids
arc['geoid'] = arc['geoid'].str[2:]
arc.head()

,geoid,census_block_group_y,census_block_group_x,city,state,zip,county,in_home_visit_date,smoke_alarms_installed__9_volt_10_year_dhh_,10_year_and_9_volt_alarms_installed,dhh_alarms_installed,pre_existing_alarms,pre_existing_alarms_tested_and_working,batteries_replaced,fire_escape_plans_made,fire_safety_checklists_completed,additional_hazard_education_conducted,additional_hazard_type,people_served,youth_served,seniors_served,veterans_military_members_and_military_family_members_served,individuals_with_disabilities_access_or_functional_needs_served
12,010010205002,32.470418,-86.424166,PRATTVILLE,AL,36066,Autauga,9/9/2016,1,1,0,0.0,0.0,0,1,1,0,None,2,0,0,0,0
18,010010208012,32.455173,-86.534591,PRATTVILLE,AL,36067,Autauga,9/30/2019,2,2,0,0.0,0.0,0,1,1,1,Tornadoes,5,0,0,1,0
23,010010208021,32.524822,-86.573009,PRATTVILLE,AL,36067,Autauga,5/4/2019,3,2,1,0.0,0.0,0,1,1,1,Other,1,0,0,0,0
24,010010208021,32.524822,-86.573009,PRATTVILLE,AL,36067,Autauga,9/27/2019,1,1,0,0.0,0.0,0,1,1,1,Tornadoes,1,0,0,0,0
27,010010208023,32.543983,-86.492100,PRATTVILLE,AL,36067,Autauga,12/2/2017,3,3,0,0.0,0.0,0,1,1,0,None,1,0,0,0,0


In [8]:
#block level
counts = arc['geoid'].value_counts()
counts_median = counts.median()
counts.describe()


count    87725.000000
mean         6.132619
std         15.250533
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max       1863.000000
Name: geoid, dtype: float64

## EDA- Geograpic Level
 repeat block Level analysis at various levels

In [9]:
# county
county_counts =  arc['geoid'].str[:7].value_counts()
print('County Level')
print(county_counts.describe())
# state 
state_counts =  arc['geoid'].str[:2].value_counts()
print('\n State Level')
print(state_counts.describe())

County Level
count    3605.000000
mean      149.232732
std       331.121218
min         1.000000
25%        10.000000
50%        47.000000
75%       148.000000
max      6579.000000
Name: geoid, dtype: float64

 State Level
count       52.000000
mean     10345.846154
std      12099.256714
min          3.000000
25%       2130.500000
50%       7135.500000
75%      12123.000000
max      51826.000000
Name: geoid, dtype: float64


In [10]:
print(arc['pre_existing_alarms'].describe())
print(arc['pre_existing_alarms_tested_and_working'].describe())

count    537984.000000
mean          1.507301
std           1.552540
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max           9.000000
Name: pre_existing_alarms, dtype: float64
count    537984.000000
mean          0.829638
std           1.336587
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           9.000000
Name: pre_existing_alarms_tested_and_working, dtype: float64


### Feature Engineering
 Create Binary variables out of the smoke_alarms_present and smoke_alarms_tested_and_working variables 
 
 We'll then create a new dataset with the aggregated number and percantage of working smoke detectors in each census geography

In [32]:


arc['pre_existing_alarms'].where(arc['pre_existing_alarms'] < 1, other = 1, inplace = True) 
arc['pre_existing_alarms_tested_and_working'].where(
                                                    arc['pre_existing_alarms_tested_and_working'] < 1,
                                                        other = 1, 
                                                        inplace = True)

# create detectors dataset 

detectors =  arc.groupby('geoid')['pre_existing_alarms'].agg({np.size ,
                                                              np.sum,
                                                              lambda x: np.sum(x)/np.size(x)* 100 })

detectors.rename({'size':'num_surveys','sum':'detectors_found_total','<lambda_0>':'detectors_found_prc'},
                 axis =1,
                 inplace = True)


d2 =  arc.groupby('geoid')['pre_existing_alarms_tested_and_working'].agg({np.sum, 
                                                                          lambda x: np.sum(x)/np.size(x)* 100 })
d2.columns = ['detectors_working_total','detectors_working_prc']

detectors = detectors.merge(d2,how = 'left', on ='geoid')
# rearrange columns 
detectors = detectors.iloc[:,[2,1,0,3,4]] 
detectors.head()

,num_surveys,detectors_found_total,detectors_found_prc,detectors_working_total,detectors_working_prc
geoid,,,,,
010010205002,1.0,0.0,0.0,0.0,0.0
010010208012,1.0,0.0,0.0,0.0,0.0
010010208021,2.0,0.0,0.0,0.0,0.0
010010208023,2.0,0.0,0.0,0.0,0.0
010010209003,3.0,0.0,0.0,0.0,0.0
